In [1]:
import numpy as np
import pandas as pd
import pyfixest as pf
from IPython.display import display, FileLink
import pathlib, requests
from tabout import MTable,  DTable, ETable, BTable, import_dta, get_var_labels, set_var_labels

df = import_dta("https://www.stata-press.com/data/r18/auto.dta")

In [2]:
get_var_labels(df)

{'make': 'Make and model',
 'price': 'Price',
 'mpg': 'Mileage (mpg)',
 'rep78': 'Repair record 1978',
 'headroom': 'Headroom (in.)',
 'trunk': 'Trunk space (cu. ft.)',
 'weight': 'Weight (lbs.)',
 'length': 'Length (in.)',
 'turn': 'Turn circle (ft.)',
 'displacement': 'Displacement (cu. in.)',
 'gear_ratio': 'Gear ratio',
 'foreign': 'Car origin'}

In [3]:
DTable(df, vars=["mpg","weight","length"], bycol=["foreign"], stats=["mean_newline_std"],
       counts_row_below=True, hide_stats=True)

In [4]:
DTable(df, vars=["mpg","weight","length"], bycol=["foreign"])

In [5]:
est=pf.feols("mpg ~ weight + length", data=df)
ETable(est)

In [6]:
import statsmodels.formula.api as smf
est2 = smf.ols("mpg ~ weight + length", data=df).fit()
ETable([est2])

In [7]:
get_var_labels(est._data)

{'make': 'Make and model',
 'price': 'Price',
 'mpg': 'Mileage (mpg)',
 'rep78': 'Repair record 1978',
 'headroom': 'Headroom (in.)',
 'trunk': 'Trunk space (cu. ft.)',
 'weight': 'Weight (lbs.)',
 'length': 'Length (in.)',
 'turn': 'Turn circle (ft.)',
 'displacement': 'Displacement (cu. in.)',
 'gear_ratio': 'Gear ratio',
 'foreign': 'Car origin'}

In [8]:
df["foreign_i"] = (df["foreign"] == "Foreign")*1
set_var_labels(df, {"foreign_i": "Foreign (indicator)"})
est_probit = smf.probit("foreign_i ~ weight + length", data=df).fit(disp=False)
ETable([est_probit], model_stats=["N","pseudo_r2","ll"])

In [10]:
# Poisson FE with a simple FE (foreign)
est_pois = pf.fepois("rep78 ~ weight + length + foreign", data=df)

# Show selected stats suitable for Poisson models
ETable(est_pois, model_stats=["N", "deviance"])


In [13]:
import pandas as pd
from linearmodels.datasets import wage_panel

data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(["nr", "year"])
data["year"] = year
print(wage_panel.DESCR)




F. Vella and M. Verbeek (1998), "Whose Wages Do Unions Raise? A Dynamic Model
of Unionism and Wage Rate Determination for Young Men," Journal of Applied
Econometrics 13, 163-183.

nr                       person identifier
year                     1980 to 1987
black                    =1 if black
exper                    labor market experience
hisp                     =1 if Hispanic
hours                    annual hours worked
married                  =1 if married
educ                     years of schooling
union                    =1 if in union
lwage                    log(wage)
expersq                  exper^2
occupation               Occupation code



In [17]:
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                   PanelOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Fri, Sep 26 2025   R-squared (Overall):              0.1893
Time:                        12:47:38   Log-likelihood                   -2982.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             72.459
                            

In [18]:
ETable(pooled_res)

TypeError: No extractor available for model type: PanelEffectsResults